# Quickstart

## Using the Compute Service

Using the notebook utilities provided by `evo-sdk-common`, we can easily interact with the Compute Service in a Jupyter notebook environment.

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id", cache_location="./notebook-data"
).login()

## JobClient

The `JobClient` wraps endpoint functionality to provide a cohesive interface to the underlying API implementation.

First we need an `APIConnector` and an `Environment` to scope the requests to a specific workspace within an organisation.

In [ ]:
environment = manager.get_environment()  # Organization ID is in the environment.
connector = manager.get_connector()  # The connector is used to authenticate requests.

## Submit a Task

The `JobClient` provides an asynchronous `submit` factory method to submit tasks to the Compute Service.
The returned `JobClient` instance can be used to monitor the task status and retrieve the results.

Tasks are referenced by `topic` and a human-readable `task` name.
Required parameters are task-dependent, so task submissions accept any mapping as the `parameters` argument.

The optional `result_type` argument can be used to specify the expected result type.
The default is `dict`, which means job results will be returned as a python dictionary.
Internally, `JobClient` uses a pydantic `TypeAdapter` to parse the job results into the specified type.

In [ ]:
from evo.compute import JobClient

break_ties = await JobClient.submit(
    connector=connector,
    org_id=environment.org_id,
    topic="geostatistics",
    task="break-ties",
    parameters={
        "source": {
            "type": "geoscience-object-reference",
            "object_reference": "geoscience_object_ref",
            "object_element": [{"type": "element", "path": "/cell_attributes/@name=attr-name"}],
        },
        "target": {
            "type": "geoscience-object-reference",
            "object_reference": "geoscience_object_ref",
            "object_element": [{"type": "element", "path": "/cell_attributes/@name=attr-name"}],
        },
        "random_seed": "123456789",
    },
    additional_headers={"API-Preview": "opt-in"},
    result_type=dict,
)
print(break_ties)  # The Job status URL is used as the string representation of a job.

## Get Job Status

`JobClient.get_status()` returns the current status of the job.

In [ ]:
response = await break_ties.get_status()
print(response)

## Get Job Results

Once the job is completed, the results can be retrieved using `JobClient.get_results()`.

If the job completed with an error, a `JobError` exception will be raised when calling `get_results()`.

In [ ]:
import json

from evo.compute.exceptions import JobError, JobPendingError

try:
    results = await break_ties.get_results()
except (JobPendingError, JobError) as e:
    print(e)
else:
    print(json.dumps(results, indent=4))

## Cancel a Job

`JobClient.cancel()` can be used to request the cancellation of a running job.
The job isn't always immediately cancelled and may take some time, so it's recommended to check the job status after calling `cancel()`.

In [ ]:
await break_ties.cancel()
print(await break_ties.get_status())

## Job Identifiers

Sometimes it may be useful to serialize and deserialize job identifiers. The job status URL may be used for this purpose.

An `APIConnector` instance is required to create a `JobClient` instance from a job status URL.
The base URL of the `APIConnector` _MUST_ match the base URL of the job status URL.

The `result_type` argument is the same as for `JobClient.submit()`.

In [ ]:
break_ties_copy = JobClient.from_url(
    connector=connector,
    url=break_ties.url,
    result_type=dict,
)
print(break_ties_copy)